---

# **Fixtures en `pytest`**

## **¿Qué es una Fixture?**
En el contexto de las pruebas, una **fixture** es un conjunto de recursos o datos que una prueba necesita para poder ejecutarse. Las fixtures te permiten preparar el entorno de prueba de manera automática, lo que simplifica el código y mejora su organización.

Por ejemplo, puedes usar una fixture para crear objetos o establecer conexiones de bases de datos que se reutilicen en varias pruebas.

### **Beneficios de las fixtures:**
- **Reusabilidad:** Permiten configurar datos o estados comunes para múltiples pruebas.
- **Aislamiento:** Garantizan que cada prueba se ejecute en un entorno limpio.
- **Automatización:** Se ejecutan automáticamente cuando son necesarias, sin intervención manual.

---

## **Uso Básico de Fixtures**

En `pytest`, puedes crear una fixture usando el decorador `@pytest.fixture`. A continuación te explico cómo funciona con un ejemplo básico.

### **Ejemplo básico:**

Vamos a crear una fixture que configure un objeto simple antes de que las pruebas se ejecuten.

```python
import pytest

class Calculadora:
    def sumar(self, a, b):
        return a + b

# Definimos una fixture
@pytest.fixture
def calculadora_fixture():
    """Crea y retorna una instancia de Calculadora."""
    return Calculadora()

# Usamos la fixture en las pruebas
def test_suma_positiva(calculadora_fixture):
    assert calculadora_fixture.sumar(3, 2) == 5

def test_suma_negativa(calculadora_fixture):
    assert calculadora_fixture.sumar(-1, -1) == -2
```

### **Explicación:**
- **`@pytest.fixture`**: Decorador que define una fixture. En este caso, la fixture `calculadora_fixture` crea una instancia de la clase `Calculadora`.
- **Uso de la fixture en las pruebas**: Las pruebas que usan la fixture deben declararla como argumento de la función (`calculadora_fixture`). `pytest` se encarga de pasar automáticamente el valor retornado por la fixture cuando se ejecutan las pruebas.
- Cada vez que una prueba llama a `calculadora_fixture`, se crea una nueva instancia de `Calculadora`, asegurando que las pruebas sean independientes entre sí.

---

## **Fixture con `yield` para Setup y Teardown**

A veces, las pruebas necesitan un proceso de "configuración" (setup) antes de ejecutarse y una "limpieza" (teardown) después de su ejecución. Para manejar esto, puedes usar `yield` en la fixture.

### **Ejemplo con setup y teardown:**

Imaginemos que tenemos una conexión a una base de datos que debe abrirse antes de la prueba y cerrarse después.

```python
import pytest

class BaseDeDatos:
    def __init__(self):
        self.conectado = False
    
    def conectar(self):
        self.conectado = True

    def desconectar(self):
        self.conectado = False

# Definimos la fixture con `yield`
@pytest.fixture
def base_datos_fixture():
    bd = BaseDeDatos()
    bd.conectar()
    yield bd  # Aquí se ejecuta la prueba
    bd.desconectar()  # Se ejecuta después de la prueba (teardown)

def test_conexion_abierta(base_datos_fixture):
    assert base_datos_fixture.conectado

def test_conexion_cerrada_despues(base_datos_fixture):
    assert base_datos_fixture.conectado
```

### **Explicación:**
- **Setup:** Antes de `yield`, el código se ejecuta para preparar el entorno de prueba. En este caso, estamos conectando a la base de datos.
- **Teardown:** El código después de `yield` se ejecuta una vez que la prueba ha terminado, en este caso desconectamos la base de datos.
- Esto asegura que incluso si la prueba falla, `pytest` ejecutará el código de limpieza automáticamente.

---

## **Alcance (Scope) de las Fixtures**

Las fixtures en `pytest` pueden tener diferentes **alcances** (scope). El alcance define cuándo se crea y destruye la fixture:

- **function** (por defecto): La fixture se ejecuta antes de cada prueba.
- **class**: La fixture se ejecuta una vez por clase de prueba.
- **module**: La fixture se ejecuta una vez por módulo.
- **session**: La fixture se ejecuta una vez por sesión completa de pruebas.

### **Ejemplo de fixture con alcance de clase:**

```python
@pytest.fixture(scope="class")
def setup_clase():
    print("\nConfigurando antes de la clase de pruebas")
    return {"data": 42}

# Usamos una clase para demostrar el uso de scope="class"
class TestClaseEjemplo:
    
    def test_primero(self, setup_clase):
        assert setup_clase["data"] == 42

    def test_segundo(self, setup_clase):
        assert setup_clase["data"] == 42
```

### **Explicación:**
- **scope="class"**: La fixture `setup_clase` se ejecutará una sola vez antes de que cualquiera de las pruebas en la clase `TestClaseEjemplo` se ejecute.
- En este caso, la fixture solo se inicializa una vez y su valor se reutiliza para ambas pruebas.

---

## **Uso de Fixtures en Conjunto**

Las fixtures pueden depender unas de otras, y `pytest` se encargará de resolver las dependencias automáticamente.

### **Ejemplo de fixture dependiente de otra fixture:**

```python
@pytest.fixture
def usuario():
    """Crea un usuario ficticio"""
    return {"nombre": "Juan", "edad": 30}

@pytest.fixture
def usuario_con_sesion(usuario):
    """Simula iniciar sesión del usuario"""
    usuario["sesion_iniciada"] = True
    return usuario

def test_sesion_iniciada(usuario_con_sesion):
    assert usuario_con_sesion["sesion_iniciada"] == True
    assert usuario_con_sesion["nombre"] == "Juan"
```

### **Explicación:**
- **`usuario_con_sesion`**: Depende de la fixture `usuario` para crear un usuario antes de marcar que la sesión ha sido iniciada.
- Esto permite encadenar fixtures para simular estados más complejos o preparaciones que dependen unas de otras.

---

## **Conclusión**

Las fixtures en `pytest` son una herramienta poderosa para gestionar el entorno de prueba, hacer que el código sea más limpio, reutilizable y eficiente. Puedes usarlas para preparar datos, configurar estados previos y realizar tareas de limpieza automáticamente después de las pruebas.

### **Puntos Clave:**
- Las fixtures permiten configurar datos o estados antes de cada prueba.
- Se pueden definir procesos de **setup** y **teardown** usando `yield`.
- El alcance de las fixtures controla cuándo se crean y destruyen (por función, clase, módulo o sesión).
- Las fixtures pueden ser dependientes, creando flujos complejos de preparación de pruebas.

---

